# Protein analysis of the longest protein of Covid 19
Three steps:
* (1) BLAST search for identifying the best aligned protein sequence
* (2) Use PDB to obtaine the geometrical information about the best aligned protein sequence from (1)
* (3) Interactive visualization with `nglview`

## (1) BLAST Search

In [1]:
from Bio import SeqIO
from Bio.Blast import NCBIWWW
from Bio import SearchIO # For parsing BLAST results


In [2]:
# Load the saved protein sequence 
protein_seq = SeqIO.read("covid_protein_seq.fasta", "fasta")
print(protein_seq.seq)

CTIVFKRVCGVSAARLTPCGTGTSTDVVYRAFDIYNDKVAGFAKFLKTNCCRFQEKDEDDNLIDSYFVVKRHTFSNYQHEETIYNLLKDCPAVAKHDFFKFRIDGDMVPHISRQRLTKYTMADLVYALRHFDEGNCDTLKEILVTYNCCDDDYFNKKDWYDFVENPDILRVYANLGERVRQALLKTVQFCDAMRNAGIVGVLTLDNQDLNGNWYDFGDFIQTTPGSGVPVVDSYYSLLMPILTLTRALTAESHVDTDLTKPYIKWDLLKYDFTEERLKLFDRYFKYWDQTYHPNCVNCLDDRCILHCANFNVLFSTVFPPTSFGPLVRKIFVDGVPFVVSTGYHFRELGVVHNQDVNLHSSRLSFKELLVYAADPAMHAASGNLLLDKRTTCFSVAALTNNVAFQTVKPGNFNKDFYDFAVSKGFFKEGSSVELKHFFFAQDGNAAISDYDYYRYNLPTMCDIRQLLFVVEVVDKYFDCYDGGCINANQVIVNNLDKSAGFPFNKWGKARLYYDSMSYEDQDALFAYTKRNVIPTITQMNLKYAISAKNRARTVAGVSICSTMTNRQFHQKLLKSIAATRGATVVIGTSKFYGGWHNMLKTVYSDVENPHLMGWDYPKCDRAMPNMLRIMASLVLARKHTTCCSLSHRFYRLANECAQVLSEMVMCGGSLYVKPGGTSSGDATTAYANSVFNICQAVTANVNALLSTDGNKIADKYVRNLQHRLYECLYRNRDVDTDFVNEFYAYLRKHFSMMILSDDAVVCFNSTYASQGLVASIKNFKSVLYYQNNVFMSEAKCWTETDLTKGPHEFCSQHTMLVKQGDDYVYLPYPDPSRILGAGCFVDDIVKTDGTLMIERFVSLAIDAYPLTKHPNQEYADVFHLYLQYIRKLHDELTGHMLDMYSVMLTNDNTSRYWEPEFYEAMYTPHTVLQAVGACVLCNSQTSLRCGACIRRPFLCCKCCYDHVISTSHKLVLSVNPYVCNAPGCDVTDVTQLYLGGMSYY

In [3]:
# BLAST search: using protein data bank (pdb) as the database
result_handle = NCBIWWW.qblast("blastp", "pdb", protein_seq.seq)

In [4]:
# Read the BLAST results => modifies the hierachy; QueryResult/Hit -> HSP
#blast_records = SearchIO.read(result_handle, 'blast-xml')
# Parse the BLAST results (recommended/conventional way)
blast_records = SearchIO.parse(result_handle, 'blast-xml')

In [5]:
# Understand the data type
print('BLAST records read via SearchIO')
print(f'Output type = {type(blast_records)}') # Check the type of the object
print(next(blast_records))

# Rewind the result_handle to the beginning
result_handle.seek(0)
# Re-parse the BLAST results to reset the iterator
blast_records = SearchIO.parse(result_handle, 'blast-xml')


BLAST records read via SearchIO
Output type = <class 'generator'>
Program: blastp (2.16.1+)
  Query: unnamed (2701)
         protein product
 Target: pdb
   Hits: ----  -----  ----------------------------------------------------------
            #  # HSP  ID + description
         ----  -----  ----------------------------------------------------------
            0      1  pdb|7D4F|A  Chain A, RNA-directed RNA polymerase [Sever...
            1      1  pdb|6YYT|A  Chain A, nsp12 [Severe acute respiratory sy...
            2      1  pdb|6XEZ|A  Chain A, RNA-directed RNA polymerase [Sever...
            3      1  pdb|9CGV|A  Chain A, RNA-directed RNA polymerase nsp12 ...
            4      1  pdb|7BW4|A  Chain A, RNA-directed RNA polymerase [Sever...
            5      1  pdb|6XQB|A  Chain A, RNA-directed RNA polymerase [Sever...
            6      1  pdb|7BV1|A  Chain A, RNA-directed RNA polymerase [Sever...
            7      1  pdb|7C2K|A  Chain A, RNA-directed RNA polymerase [Sever.

### (1.1) BLAST SearchIO's output
* [Official document](https://biopython.org/docs/1.76/api/Bio.SearchIO.BlastIO.html#submodules)
* The output is hierarchical; 3-level containers
    * (1) QueryResult (<= this level will be flattened, if using SearchIO.read)
    * (2) Hit 
        * Hit ID = database|accession|chain
        * (e.g.) 'pdb|7D4F|A' means: database = pdb, accession(ID in PDB) = 7D4F, chain = A
    * (3) HSP(= High-scoring Sequencing Pair)
* Source code on [Hit Class](https://github.com/biopython/biopython/blob/master/Bio/SearchIO/_model/hit.py)
* (??) Are the query results sorted according to the aligment-level? i.e. the first result is the best-aligned result? 

In [6]:
best_aligned_id = None
best_aligned_score = 0.0

for query_result in blast_records:
    print(f"\nQuery ID: {query_result.id} (Description: {query_result.description})")
    print(f"Found {len(query_result)} hits")
    
    # Loop through all hits for this query
    for hit in query_result:
        print(f"\nHit ID: {hit.id}")
        print(f"Hit description: {hit.description}")
        print(f"Number of HSPs: {len(hit)}")
        
        # Now loop through HSPs for this hit
        for hsp in hit:
            print(f"\n  HSP E-value: {hsp.evalue}")
            print(f"  Bit score: {hsp.bitscore}")
            print(f"  Query range: {hsp.query_start}-{hsp.query_end}")
            print(f"  Hit range: {hsp.hit_start}-{hsp.hit_end}")

            # Check if this HSP has the best score so far
            if hsp.bitscore > best_aligned_score:
                best_aligned_score = hsp.bitscore
                best_aligned_id = hit.id
                print(f"  New best alignment found with ID: {best_aligned_id} and score: {best_aligned_score}")

# Rewind the result_handle to the beginning
result_handle.seek(0)

# Re-parse the BLAST results to reset the iterator
blast_records = SearchIO.parse(result_handle, 'blast-xml')



Query ID: unnamed (Description: protein product)
Found 50 hits

Hit ID: pdb|7D4F|A
Hit description: Chain A, RNA-directed RNA polymerase [Severe acute respiratory syndrome coronavirus 2]
Number of HSPs: 1

  HSP E-value: 0.0
  Bit score: 1938.7
  Query range: 4-930
  Hit range: 8-934
  New best alignment found with ID: pdb|7D4F|A and score: 1938.7

Hit ID: pdb|6YYT|A
Hit description: Chain A, nsp12 [Severe acute respiratory syndrome coronavirus 2]
Number of HSPs: 1

  HSP E-value: 0.0
  Bit score: 1938.31
  Query range: 4-929
  Hit range: 10-935

Hit ID: pdb|6XEZ|A
Hit description: Chain A, RNA-directed RNA polymerase [Severe acute respiratory syndrome coronavirus 2]
Number of HSPs: 1

  HSP E-value: 0.0
  Bit score: 1937.92
  Query range: 4-929
  Hit range: 7-932

Hit ID: pdb|9CGV|A
Hit description: Chain A, RNA-directed RNA polymerase nsp12 [Severe acute respiratory syndrome coronavirus 2]
Number of HSPs: 1

  HSP E-value: 0.0
  Bit score: 1937.54
  Query range: 4-929
  Hit range: 2

In [7]:
best_aligned_id
best_hit = query_result[best_aligned_id]
print(f"Match to {best_hit.description}") 

Match to Chain A, RNA-directed RNA polymerase [Severe acute respiratory syndrome coronavirus 2]


#### Remark
* Apparently the best hit ID **"pdb|7D4F|A"** consists of three identical protein chain (called **homotrimer**)
* However, these three chains are labeled differently as Chain A, B, and C to reflect the differences in their 3D structure
* The BLAST hit to **"pdb|7D4F|A"** means the query aligns with one copy of the spike protein in the trimer.
* Chains B and C would show identical alignments if tested (but BLAST typically reports only one representative).

## (2) Get the structure information from PDB

In [8]:
this_id = "pdb|6YYT|A".split("|")[1] # Here, using the second best alignment
pdb_link = f'https://files.rcsb.org/download/{this_id}.pdb'

In [9]:
# Download the PDB file of this ID
import urllib.request

# Download the PDB file of this ID
urllib.request.urlretrieve(pdb_link, f"{this_id}.pdb")

('6YYT.pdb', <http.client.HTTPMessage at 0x10b44dbf0>)

In [10]:
from Bio.PDB import PDBParser 

In [11]:
# Steps: (1) parse the PDF file, (2) get_structure()
parser = PDBParser()
structure = parser.get_structure(this_id, f'{this_id}.pdb') 

/Users/sayakokodera/miniforge3/envs/bioinf/lib/python3.13/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 12060.
  warnings.warn(


In [12]:
# Check how many chains are there
print(f'Total number of chains: {len(structure[0])}')
for chain in structure[0]:
    print(f'Chain ID: {chain.id}')
    print(f'Chain length: {len(chain)}')
    print('---')

Total number of chains: 8
Chain ID: A
Chain length: 855
---
Chain ID: B
Chain length: 186
---
Chain ID: C
Chain length: 73
---
Chain ID: D
Chain length: 186
---
Chain ID: P
Chain length: 14
---
Chain ID: Q
Chain length: 14
---
Chain ID: T
Chain length: 12
---
Chain ID: U
Chain length: 14
---


In [13]:
# Check if A and B chains are identical 
chain_a_seq = "".join([residue.resname for residue in structure[0]["A"]])
chain_b_seq = "".join([residue.resname for residue in structure[0]["B"]])
print(chain_a_seq == chain_b_seq)  # True if identical

False


## (3) Visualize

In [14]:
import nglview as nv

In [15]:
#nv.show_biopython(structure, gui=True)

### Observations
* I can see the followin chains
    * A = red ribbon = polymerase 
    * B = orange ribbon = viral protein 
    * C = orange ribbon = viral protein
    * D = yellow ribbon = viral protein 
    * U, Q = blue helics = RNA template strand and its product strand
* (??) where are Chain P and T??